# Texto para Áudio

Até aqui, trabalhamos sempre passando áudio como entrada, e recebendo texto como saída. Mas há outros modelos que fazem o caminho contrário: passamos texto e recebemos áudio.

Vamos explorar esta funcionalidade com um modelo feito para a língua portuguesa: https://huggingface.co/facebook/mms-tts-por

In [1]:
from transformers import pipeline

modelo = "facebook/mms-tts-por"
leitor = pipeline('text-to-speech', model=modelo)

Some weights of the model checkpoint at facebook/mms-tts-por were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [2]:
import time

texto = 'Olá, meu nome é Juliano e estou aprendendo Python!'

inicio = time.time()
fala = leitor(texto)
final = time.time()

print(f'levou {final - inicio:.02f} segundos para gerar o áudio')
print(fala)

levou 5.43 segundos para gerar o áudio
{'audio': array([[-3.0458840e-03, -2.3931328e-03, -2.4038469e-03, ...,
         1.8329105e-04,  2.8829806e-04,  6.5238470e-05]], dtype=float32), 'sampling_rate': 16000}


In [3]:
import IPython

IPython.display.Audio(data=fala['audio'], rate=fala['sampling_rate'])

Este é um modelo simples, que tenta traduzir as letras e sílabas em sons. Porém ele geralmente falha quando há palavras estrangeiras. Vamos tentar usar um modelo mais avançado (e mais lento).

## Modelo Bark

Vamos explorar outro modelo: https://huggingface.co/suno/bark-small

In [20]:
modelo = 'suno/bark-small'
leitor = pipeline('text-to-speech', model=modelo, forward_params={'max_new_tokens': 50})

In [ ]:
texto = 'Olá, meu nome é Juliano e estou aprendendo Python!'

inicio = time.time()
fala = leitor(texto)
final = time.time()

print(f'levou {final - inicio:.03f} segundos para gerar o áudio')
print(fala)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
IPython.display.Audio(data=fala['audio'], rate=fala['sampling_rate'])

## Otimizando e configurando o modelo

O modelo Bark é melhor que o anterior, mas ele leva um tempo considerável para executar. Vamos tentar otimizá-lo!

### Instalando bibliotecas de otimização

O Hugging Face possui outras bibliotecas além de `transformers` e `datasets`. Vamos instalar as seguintes bibliotecas de otimização:

```bash
pip install accelerate
pip install optimum
```

**Atenção**: praticamente todas estas otimizações utilizam a placa de vídeo (GPU) do computador. Portanto, só é possível acelerar a execução do modelo se:
- Você tiver uma placa de vídeo no seu computador.
- Você tiver instalado o PyTorch com suporte para o CUDA (a biblioteca responsável por "conversar" com a GPU). Veja detalhes de instalação em: https://pytorch.org/

### Testando com otimizações

In [15]:
import torch

modelo = "suno/bark-small"

leitor = pipeline('text-to-speech', model=modelo, model_kwargs={'torch_dtype': torch.float16}, forward_params={'max_new_tokens': 50})
device = "cuda" if torch.cuda.is_available() else "cpu"
leitor.model = leitor.model.to(device)
leitor.model = leitor.model.to_bettertransformer()
leitor.model.enable_cpu_offload()

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [16]:
texto = 'Olá, meu nome é Juliano e estou aprendendo Python!'

inicio = time.time()
fala = leitor(texto)
final = time.time()

print(f'levou {final - inicio:.03f} segundos para gerar o áudio')
print(fala)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
IPython.display.Audio(data=fala['audio'], rate=fala['sampling_rate'])

## Selecionando a voz do modelo

Os modelos Bark possuem diferentes vozes, que você pode conferir [neste link aqui](https://suno-ai.notion.site/8b8e8749ed514b0cbf3f699013548683?v=bc67cff786b04b50b3ceb756fd05f68c).

Se não passarmos uma voz específica, ele seleciona uma voz conforme a língua. Contudo, se já sabemos qual a língua do texto, faz mais sentido passarmos uma voz explicitamente.

Não conseguimos passar o argumento de escolha de voz diretamente pelo pipeline. Por isso, vamos precisar fazer a predição "manualmente":

In [ ]:
import torch
from transformers import AutoProcessor, AutoModel

modelo = 'suno/bark-small'

processador = AutoProcessor.from_pretrained(modelo,  torch_dtype=torch.float16, max_new_tokens=50)
leitor = AutoModel.from_pretrained(modelo)

# Otimizações
device = "cuda" if torch.cuda.is_available() else "cpu"
leitor = leitor.to(device)
leitor = leitor.to_bettertransformer()
leitor.enable_cpu_offload()

In [ ]:
texto = 'Olá, meu nome é Juliano e estou aprendendo Python!'
voz = 'v2/pt_speaker_1'

inicio = time.time()
inputs = processador(texto, voice_preset=voz).to(device)
vetor_audio = leitor.generate(**inputs)
fala = {
    'audio': vetor_audio.cpu().numpy(),
    'sampling_rate': leitor.generation_config.sample_rate,
}
final = time.time()

print(f'levou {final - inicio:.03f} segundos para gerar o áudio')
print(fala)

In [ ]:
IPython.display.Audio(data=fala['audio'], rate=fala['sampling_rate'])

## Processamento em paralelo

Podemos passar uma lista de inputs para processar em paralelo (a diferença é mais perceptível em computadores / GPUs melhores):

In [ ]:
modelo = 'suno/bark-small'

processador = AutoProcessor.from_pretrained(modelo, torch_dtype=torch.float16, max_new_tokens=50)
leitor = AutoModel.from_pretrained(modelo)

# Otimizações
device = "cuda" if torch.cuda.is_available() else "cpu"
leitor = leitor.to(device)
leitor = leitor.to_bettertransformer()
leitor.enable_cpu_offload()


In [ ]:
textos = [
    'Olá, meu nome é Juliano e estou aprendendo Python.',
    'Hoje pela manhã, fui passear pela cidade e aproveitar o dia de sol.',
    'Agora que voltei para casa, consigo focar nos meus estudos de Python.',
]
voz = 'v2/pt_speaker_1'

inicio = time.time()
inputs = processador(textos, voice_preset=voz).to(device)
vetores = leitor.generate(**inputs)
falas = [
    {
        'audio': vetor_audio.cpu().numpy(),
        'sampling_rate': leitor.generation_config.sample_rate,
    }
    for vetor_audio in vetores 
]
final = time.time()

print(f'levou {final - inicio:.03f} segundos para gerar o áudio')
print(falas)

In [11]:
for fala in falas:
    display(IPython.display.Audio(data=fala['audio'], rate=fala['sampling_rate']))

## Outros modelos

Há diversos outros modelos para produzir sons, músicas, etc a partir de texto. Vamos explorar mais um através da Inference API:

In [37]:
import requests

modelo = 'facebook/musicgen-small'
url = f"https://api-inference.huggingface.co/models/{modelo}"

json = {
    # 'inputs': 'A chill mysterious beat for a detective podcast intro',
    'inputs': '80s music with synths and a happy beat',
}

response = requests.post(url, json=json)

In [38]:
print(response)

<Response [429]>


In [39]:
from pathlib import Path

pasta_musicas = Path('audios/musicas')
pasta_musicas.mkdir(exist_ok=True, parents=True)
                    
if response.content:
    with open(pasta_musicas / 'music.wav', 'wb') as arquivo_musica:
        arquivo_musica.write(response.content)